![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/reader/SparkNLP_HTML_Reader_Demo.ipynb)

# Introducing HTML reader in SparkNLP
This notebook showcases the newly added  `sparknlp.read().html()` method in Spark NLP that parses HTML content from both local files and real-time URLs into a Spark DataFrame.

**Key Features:**
- Ability to parse HTML from local directories and URLs.
- Versatile support for varied data ingestion scenarios.

## Setup and Initialization
Let's keep in mind a few things before we start 😊

Support for reading html files was introduced in Spark NLP 5.5.2. Please make sure you have upgraded to the latest Spark NLP release.

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

For local files example we will download a couple of HTML files from Spark NLP Github repo:

In [6]:
!mkdir html-files
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/html/example-10k.html -P html-files
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/html/fake-html.html -P html-files

--2025-03-05 23:21:42--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/html/example-10k.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2456707 (2.3M) [text/plain]
Saving to: ‘html-files/example-10k.html’

example-10k.html    100%[===================>]   2.34M  --.-KB/s    in 0.08s   

2025-03-05 23:21:43 (30.6 MB/s) - ‘html-files/example-10k.html’ saved [2456707/2456707]

--2025-03-05 23:21:43--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/html/fake-html.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

## Parsing HTML from Local Files
Use the `html()` method to parse HTML content from local directories.

In [7]:
import sparknlp
html_df = sparknlp.read().html("./html-files")

html_df.show()

+--------------------+--------------------+
|                path|                html|
+--------------------+--------------------+
|file:/content/htm...|[{Title, UNITED S...|
|file:/content/htm...|[{Title, My First...|
+--------------------+--------------------+



## Parsing HTML from Real-Time URLs
Use the `html()` method to fetch and parse HTML content from a URL or a set of URLs.

In [8]:
html_df = sparknlp.read().html("https://example.com/")
html_df.show(truncate=False)

+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|url                 |html                                                                                                                                                                                                                                                                                                                   |
+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
html_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- html: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- elementType: string (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)



In [10]:
htmls_df = sparknlp.read().html(["https://www.wikipedia.org", "https://example.com/"])
htmls_df.show()

+--------------------+--------------------+
|                 url|                html|
+--------------------+--------------------+
|https://www.wikip...|[{Title, Wikipedi...|
|https://example.com/|[{Title, Example ...|
+--------------------+--------------------+



### Configuration Parameters

You can customize the font size used to identify paragraphs that should be treated as titles. By default, the font size is set to 16. However, if your HTML files require a different configuration, you can adjust this parameter accordingly. The example below demonstrates how to modify and work with this setting:

In [11]:
params = {"titleFontSize": "12"}
html_df = sparknlp.read(params).html("./html-files/fake-html.html")
html_df.select("html").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|html                                                                                                                                                                                                                                                                                                                                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

You can access the raw content of the file using the `storeContent` parameter. This parameter was added in Spark NLP 6.0.0

In [12]:
params = {"storeContent": "true"}
html_df = sparknlp.read(params).html("./html-files/fake-html.html")
html_df.show()

+--------------------+--------------------+--------------------+
|                path|             content|                html|
+--------------------+--------------------+--------------------+
|file:/content/htm...|<!DOCTYPE html>\n...|[{Title, My First...|
+--------------------+--------------------+--------------------+

